# Modelowanie liczby roszczeń

### Generowanie niejednorodnego procesu Poissona metodą przerzedzania

In [7]:
import numpy as np
import pandas as pd

In [22]:
def nhpp(lambda_func,lambda_0,lambda_func_params, t_max):
    """
    Funkcja generuje niejednorodny proces Poissona metodą przerzerzedzania.

    Argumenty:
        lambda_func: funkcja
            Funkcja intensywności
        lambda_0: float
            Intensywność bazowa procesu
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności
        t_max: float
            Maksymalny czas symulacji
        lambda_max: float
            Ograniczenie górne funkcji lambda_func
    Zwraca:
        numpy.ndarray
            Tablica czasów.
    """

    S = 0
    N = 0
    S_tab = [0]
    lambda_max = lambda_func(0,lambda_0,lambda_func_params)[1]
    
    while S < t_max:
        U = np.random.uniform(0,1)
        S=S+(-1/lambda_max)*np.log(U)
        if S > t_max:
            break
        U = np.random.uniform(0,1)
        if U <= lambda_func(S,lambda_0,lambda_func_params)[0]/lambda_max:
            N = N + 1
            S_tab.append(S)

    return N, S_tab

In [18]:
def sin_int_fun(t,lambda_0,lambda_func_params):
    """
    Sinusoidalna funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_0: float
            Intensywność bazowa procesu
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności.
            W tym przypadku listę zawierającą: [lambda_1, T, phi]
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0+lambda_1.    
    """
    return lambda_0+lambda_func_params[0]*np.sin(2*np.pi/lambda_func_params[1]+lambda_func_params[2]), lambda_0+lambda_func_params[0]

In [19]:
def exp_int_fun(t,lambda_0,lambda_func_params):
    """
    Wykładnicza funkcja intensywności.

    Argumenty:
        t: float
            Funkcja intensywności
        lambda_0: float
            Intensywność bazowa procesu
        lambda_func_params: tab
            Tablica zawierająca parametry funkcji intensywności. 
            W tym przypadku listę jednoelemnetową zawierającą parametr lambda.
    Zwraca:
        float
            Wartość funkcji intensywności.
        float
            Górne ograniczenie funkcji.
            W tym przypadku lambda_0/lambda.
    """
    return lambda_0*np.exp(-lambda_func_params[0]*t), lambda_0*lambda_func_params[0]

In [23]:
nhpp(sin_int_fun,1,[3,1,0],1000)

(1031,
 [0,
  0.6271345056863997,
  0.7104504076689033,
  1.1765733179600468,
  2.0696870321704215,
  3.5479110799001177,
  4.724707338749178,
  4.989732753733382,
  5.021746555890064,
  5.743465937153971,
  5.743547120916564,
  7.079194479140354,
  7.709288883121805,
  7.9925906429958085,
  10.675109253191403,
  12.591358540614683,
  14.061563508886868,
  14.471816465071646,
  15.395396345812088,
  18.5761490164754,
  18.81931119354409,
  20.404238021694837,
  20.86712065502881,
  21.38227112665786,
  22.600353700600277,
  22.802387856027206,
  25.710741906435036,
  27.40185653200364,
  28.62455434463561,
  29.069777203181484,
  31.769776302826557,
  32.3753625145057,
  32.86494718721214,
  33.75927047962487,
  34.9573464248383,
  35.67562126631976,
  35.915277360402364,
  35.975823138061834,
  37.32663839901013,
  38.11269212858995,
  38.216922011174745,
  39.4600056032611,
  39.53837321372968,
  40.212874742596746,
  41.11837713797338,
  41.328983855762154,
  42.58523456221207,
  45

In [32]:
nhpp(exp_int_fun,2,[1/10],10000)

(5,
 [0,
  7.131305365570494,
  14.25212321616105,
  21.704430797235787,
  26.068299180708898,
  26.566511929881898])